https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [35]:
def readTrain():
    train = pd.read_csv("SPY.csv")
    return train

readTrain()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.968700,43.968700,43.750000,43.937500,27.112253,1003200
1,1993-02-01,43.968700,44.250000,43.968700,44.250000,27.305082,480500
2,1993-02-02,44.218700,44.375000,44.125000,44.343700,27.362904,201300
3,1993-02-03,44.406200,44.843700,44.375000,44.812500,27.652185,529400
4,1993-02-04,44.968700,45.093700,44.468700,45.000000,27.767893,531500
5,1993-02-05,44.968700,45.062500,44.718700,44.968700,27.748577,492100
6,1993-02-08,44.968700,45.125000,44.906200,44.968700,27.748577,596100
7,1993-02-09,44.812500,44.812500,44.562500,44.656200,27.555738,122100
8,1993-02-10,44.656200,44.750000,44.531200,44.718700,27.594305,379600
9,1993-02-11,44.781200,45.125000,44.781200,44.937500,27.729336,19500


In [9]:
def augFeatures(train):
    train["Date"] = pd.to_datetime(train["Date"])
    train["year"] = train["Date"].dt.year
    train["month"] = train["Date"].dt.month
    train["date"] = train["Date"].dt.day
    train["day"] = train["Date"].dt.dayofweek
    return train

In [8]:
def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [10]:
def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
    return np.array(X_train), np.array(Y_train)

In [16]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X, Y

In [18]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [25]:

# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
print(X_train.shape)
# X_trian: (5710, 30, 10)
# Y_train: (5710, 5, 1)
# X_val: (634, 30, 10)
# Y_val: (634, 5, 1)


(5680, 30, 10)


# One to one model  
input shape:  
(5710, 1, 10)  
output shape:  
(5710, 1, 1)  

In [26]:
def buildOneToOneModel(shape):
    #print(shape)
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [29]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]
print(X_train.shape)
print(Y_train.shape)
model = buildOneToOneModel(X_train.shape)
# callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
# model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


(5710, 1, 10)
(5710, 1, 1)


/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


# Many to one model  
input shape:  
(5710, 1, 10)  
output shape:  
(5710, 1, 1)  

In [31]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [33]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)


print(X_train.shape)
print(Y_train.shape)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

(5684, 30, 10)
(5684, 1)


/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/1000
5684/5684 [==============================] - 1s 252us/step - loss: 0.1084 - val_loss: 0.1774
Epoch 2/1000
5684/5684 [==============================] - 1s 108us/step - loss: 0.0170 - val_loss: 0.0331
Epoch 3/1000
5684/5684 [==============================] - 1s 95us/step - loss: 0.0030 - val_loss: 0.0114
Epoch 4/1000
5684/5684 [==============================] - 1s 100us/step - loss: 0.0011 - val_loss: 0.0054
Epoch 5/1000
5684/5684 [==============================] - 1s 94us/step

5684/5684 [==============================] - 1s 92us/step - loss: 4.5646e-05 - val_loss: 6.5030e-05
Epoch 68/1000
5684/5684 [==============================] - 1s 90us/step - loss: 4.4726e-05 - val_loss: 5.3845e-05
Epoch 69/1000
5684/5684 [==============================] - 0s 86us/step - loss: 4.3479e-05 - val_loss: 4.1608e-05
Epoch 70/1000
5684/5684 [==============================] - 0s 88us/step - loss: 4.3938e-05 - val_loss: 2.7577e-05
Epoch 71/1000
5684/5684 [==============================] - 0s 88us/step - loss: 4.4320e-05 - val_loss: 5.7346e-05
Epoch 72/1000
5684/5684 [==============================] - 0s 88us/step - loss: 4.4202e-05 - val_loss: 3.0143e-05
Epoch 73/1000
5684/5684 [==============================] - 0s 88us/step - loss: 4.2781e-05 - val_loss: 2.6544e-05
Epoch 74/1000
5684/5684 [==============================] - 1s 88us/step - loss: 4.3055e-05 - val_loss: 2.7011e-05
Epoch 75/1000
5684/5684 [==============================] - 0s 86us/step - loss: 4.1709e-05 - val_loss:

5684/5684 [==============================] - 1s 122us/step - loss: 3.2466e-05 - val_loss: 8.9708e-06
Epoch 139/1000
5684/5684 [==============================] - 1s 117us/step - loss: 3.1310e-05 - val_loss: 5.3893e-06
Epoch 140/1000
5684/5684 [==============================] - 1s 120us/step - loss: 3.1377e-05 - val_loss: 7.8616e-06
Epoch 141/1000
5684/5684 [==============================] - 1s 97us/step - loss: 3.4203e-05 - val_loss: 4.3101e-06
Epoch 142/1000
5684/5684 [==============================] - 1s 105us/step - loss: 3.2004e-05 - val_loss: 1.9826e-05
Epoch 143/1000
5684/5684 [==============================] - 1s 90us/step - loss: 3.1695e-05 - val_loss: 1.7044e-05
Epoch 144/1000
5684/5684 [==============================] - 1s 101us/step - loss: 3.1212e-05 - val_loss: 1.3673e-05
Epoch 145/1000
5684/5684 [==============================] - 1s 103us/step - loss: 3.2424e-05 - val_loss: 1.5416e-05
Epoch 146/1000
5684/5684 [==============================] - 1s 98us/step - loss: 3.1357e-